## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [8]:
# prompt 생성
prompt = prompt_template.format(country="캐나다")
prompt

'캐나다의 수도는 어디인가요?'

In [9]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [15]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

In [16]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AFA7949710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AFA641D450>, root_client=<openai.OpenAI object at 0x000001AFA5D54E50>, root_async_client=<openai.AsyncOpenAI object at 0x000001AFA794A3D0>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [24]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how":"5살짜리 어린 아이에게도 이해할 수 있게"}

In [26]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
answer = chain.invoke(input)

In [30]:
answer.content
answer.response_metadata

{'token_usage': {'completion_tokens': 214,
  'prompt_tokens': 34,
  'total_tokens': 248,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_709714d124',
 'finish_reason': 'stop',
 'logprobs': None}

아래는 스트리밍을 출력하는 예시 입니다.

In [31]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

좋아! 인공지능 모델의 학습 원리를 아주 쉽게 설명해줄게.

인공지능 모델은 마치 똑똑한 로봇 친구 같아. 이 로봇 친구는 많은 그림과 이야기를 보고 배우면서 점점 더 똑똑해져. 

1. **배우기**: 처음에 로봇 친구는 많은 그림을 보여줘. 예를 들어, 강아지 그림과 고양이 그림을 보여주면서 "이건 강아지야!" 또는 "이건 고양이야!"라고 말해줘.

2. **기억하기**: 로봇 친구는 이렇게 배운 것들을 머릿속에 잘 기억해. 그래서 다음에 강아지 그림을 보면 "아! 이건 강아지야!"라고 말할 수 있어.

3. **연습하기**: 로봇 친구는 계속 연습해. 더 많은 그림을 보고, 더 많은 이야기를 듣고, 점점 더 잘 맞추게 돼.

4. **틀릴 때 배우기**: 만약 로봇 친구가 "이건 고양이야!"라고 말했는데 사실은 강아지였다면, "아! 이건 강아지였구나!"라고 배우고 다시는 그렇게 말하지 않도록 해.

이렇게 로봇 친구는 계속 배우고 연습하면서 점점 더 똑똑해지는 거야! 그래서 나중에는 우리가 질문을 하면 잘 대답할 수 있게 되는 거지. 이해했니?

### 출력파서(Output Parser)


In [32]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [33]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [35]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how":"5살짜리 어린 아이에게도 이해할 수 있게"}
chain.invoke(input)

'좋아! 인공지능 모델의 학습 원리를 아주 쉽게 설명해줄게.\n\n우리가 새로운 것을 배우는 방법을 생각해보자. 예를 들어, 너가 사과를 처음 봤을 때, 사과가 빨갛고 둥글고 맛있다는 걸 알게 되지? 그럼 다음에 사과를 보면 "아! 이건 사과야!" 하고 알 수 있어.\n\n인공지능도 비슷하게 배워. 처음에는 많은 사진이나 정보를 보여줘. 예를 들어, 사과 사진을 많이 보여주고 "이건 사과야!"라고 말해줘. 그러면 인공지능은 사과의 모양과 색깔을 기억하게 돼.\n\n그리고 나중에 새로운 사진을 보여주면, "이건 사과일까?" 하고 생각해. 만약 맞으면 잘했어! 틀리면 "아, 이건 사과가 아니구나!" 하고 다시 배워. 이렇게 계속 반복하면서 점점 더 똑똑해지는 거야.\n\n그래서 인공지능은 많은 정보를 보고 배우면서, 우리가 사물을 알아보는 것처럼 스스로도 알아보는 능력을 키우는 거야! 이해했니?'

In [36]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

좋아! 인공지능 모델의 학습 원리를 아주 쉽게 설명해볼게.

인공지능 모델은 마치 똑똑한 로봇 친구 같아. 이 로봇 친구는 많은 그림과 이야기를 보고 배우면서 점점 더 똑똑해져. 

1. **보기**: 로봇 친구는 많은 그림이나 사진을 봐. 예를 들어, 강아지 사진을 많이 보여주면, "아! 이건 강아지구나!" 하고 알아차려.

2. **연습하기**: 로봇 친구는 강아지 사진을 보고 "이건 강아지야!"라고 말하는 연습을 해. 처음에는 잘 못할 수도 있지만, 계속 연습하면 점점 잘하게 돼.

3. **틀린 것 고치기**: 만약 로봇 친구가 "이건 고양이야!"라고 잘못 말하면, 누군가가 "아니야, 이건 강아지야!"라고 알려줘. 그러면 로봇 친구는 그걸 기억하고 다음에는 잘 말할 수 있어.

4. **점점 똑똑해지기**: 이렇게 많은 그림과 이야기를 보고 연습하고 고치면서 로봇 친구는 점점 더 똑똑해져. 나중에는 새로운 강아지 사진을 보고도 "이건 강아지야!"라고 잘 말할 수 있게 돼.

그래서 인공지능 모델은 많은 것을 보고 배우면서 점점 더 똑똑해지는 거야! 이해했니?

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [37]:
# 개행 문자 템플릿 작성성
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [38]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [39]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Customer:** Hi there! Could I see the menu, please?  
  **Waiter:** Of course! Here you go.  
  **Customer:** Thank you! What do you recommend?  
  **Waiter:** Our steak is very popular and the pasta is also a great choice.  
  **Customer:** I think I'll have the steak, please. Can I get it medium rare?  
  **Waiter:** Absolutely! Would you like any side dishes with that?  
  **Customer:** Yes, I'll have a side salad and mashed potatoes, please.  
  **Waiter:** Great choice! Would you like anything to drink?  
  **Customer:** Just water, please.  
  **Waiter:** Alright! Your order will be ready shortly.

- 한글 해석:  
  **손님:** 안녕하세요! 메뉴 좀 볼 수 있을까요?  
  **웨이터:** 물론이죠! 여기 있습니다.  
  **손님:** 감사합니다! 추천해 주실 만한 게 있나요?  
  **웨이터:** 저희 스테이크가 매우 인기 있고 파스타도 훌륭한 선택이에요.  
  **손님:** 그럼 스테이크로 할게요. 미디엄 레어로 말이죠.  
  **웨이터:** 알겠습니다! 사이드 메뉴는 어떤 걸로 드릴까요?  
  **손님:** 네, 사이드 샐러드와 매시드 포테이토로 주세요.  
  **웨이터:** 좋은 선택이에요! 음료수는 어떻게 하시겠어요?  
  **손님:** 그냥 물로 주세요.  
  **웨이터:** 알겠습니다! 주문은 곧 준비될 거예요.


In [40]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hello! Welcome to our restaurant. Can I take your order?
B: Yes, please. I would like the grilled salmon and a side salad.
A: Great choice! Would you like any drinks with that?
B: Yes, I’ll have a glass of white wine, please.
A: Perfect! Your order will be ready shortly.

- 한글 해석:
A: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 주문하시겠어요?
B: 네, 물론이요. 구운 연어와 사이드 샐러드를 주문하고 싶어요.
A: 좋은 선택이에요! 음료수는 필요하신가요?
B: 네, 화이트 와인 한 잔 주세요.
A: 완벽해요! 주문하신 음식은 곧 준비될 거예요.

In [41]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Customer: Hi, I’d like to place an order for a pizza, please.
  - Pizza Place: Sure! What size would you like?
  - Customer: I’ll have a large, please. Can I get half pepperoni and half veggie?
  - Pizza Place: Of course! Would you like any drinks or sides with that?
  - Customer: Yes, I’ll take a 2-liter soda and some garlic bread.
  - Pizza Place: Great choice! Can I have your address for delivery?
  - Customer: Sure, it’s 123 Main Street, Apartment 4B.
  - Pizza Place: Thank you! Your order will be delivered in about 30 minutes. 
  - Customer: Awesome, thank you!

- 한글 해석:
  - 고객: 안녕하세요, 피자 주문하고 싶어요.
  - 피자 가게: 알겠습니다! 어떤 사이즈로 하시겠어요?
  - 고객: 큰 사이즈로 주세요. 반은 페퍼로니, 반은 채소로 해주세요.
  - 피자 가게: 물론입니다! 음료나 사이드도 필요하세요?
  - 고객: 네, 2리터 소다와 마늘빵 하나 주세요.
  - 피자 가게: 좋은 선택입니다! 배달 주소를 주실 수 있나요?
  - 고객: 네, 123 메인 스트리트, 아파트 4B입니다.
  - 피자 가게: 감사합니다! 주문은 약 30분 후에 배달됩니다.
  - 고객: 좋네요, 감사합니다!

In [42]:
from langchain_teddynote.messages import stream_response

answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})

In [43]:
stream_response(answer)

- 영어 회화:  
  **Waiter:** Good evening! How can I help you today?  
  **You:** Hi! I’d like to see the menu, please.  
  **Waiter:** Of course! Here it is. Do you have any questions about the dishes?  
  **You:** Yes, I’d like to know what the chef’s special is.  
  **Waiter:** The chef’s special today is grilled salmon with seasonal vegetables.  
  **You:** That sounds great! I’ll have the grilled salmon, please.  
  **Waiter:** Would you like any appetizers or drinks with that?  
  **You:** Yes, I’ll have a Caesar salad and a glass of white wine, please.  
  **Waiter:** Excellent choice! I’ll get that order in for you.  

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 어떻게 도와드릴까요?  
  **당신:** 안녕하세요! 메뉴를 보여주실 수 있나요?  
  **웨이터:** 물론이죠! 여기 있습니다. 요리에 대해 질문이 있으신가요?  
  **당신:** 네, 오늘 셰프의 추천 요리가 무엇인지 알고 싶습니다.  
  **웨이터:** 오늘의 셰프 특별 메뉴는 구운 연어와 제철 채소입니다.  
  **당신:** 훌륭하네요! 구운 연어로 주세요.  
  **웨이터:** 추가 주문이나 음료는 필요하신가요?  
  **당신:** 네, 시저 샐러드와 화이트 와인 한 잔 주세요.  
  **웨이터:** 훌륭한 선택입니다! 주문을 넣도록 하겠습니다.